In [1]:
""""Code to format info on movies in a csv file for publication."""

# built-in libraries
import csv
import datetime
from email.message import EmailMessage
import os.path
import re
import smtplib
import sqlite3
import sys

# installed with pip
import pandas as pd
#from sqlalchemy import create_engine

# Create list of dictionaries for each row.
listings = []
scrapedir = '/Users/Doug/workspace/cinema/scrapes/'
scrapefile = 'fandango-2019-06-27.csv'
scrape_path = scrapedir + scrapefile
with open(scrape_path, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        layout_dct = {
            'addy': row[0],
            'theater': row[1],
            'title': row[2],
            'mpaa': row[3],
            'stars': row[4],
            'times': row[5],
            'format': row[6],
            'synopsis': row[7],
            'date': row[8],
            'city': row[9]
        }
        listings.append(layout_dct)
        
del listings[0] #remove header row   

# Create Pandas dataframe for listings
df = pd.DataFrame(listings)
# Remove some duplicate listings
df = df[df.city != "Belleview, FL"]
df = df[df.city != "High Springs, FL"]
df = df[(df.city != "The Villages, FL") & (df.theater != "Belleview Cinemas")]
df = df[(df.city != "The Villages, FL") & (df.theater != "Old Mill Playhouse")]
df = df[(df.city != "The Villages, FL") & (df.theater != "Old Mill Playhouse")]
times = (df.times.str.split('|', expand=True).rename(columns=lambda x: f"time_{x+1}"))
times = times.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
times = times.apply(lambda x: x.str.replace('p', ' p.m.') if x.dtype == "object" else x)
times = times.apply(lambda x: x.str.replace('a', ' a.m.') if x.dtype == "object" else x)
times = times.apply(lambda x: x.str.replace(':00', '') if x.dtype == "object" else x)
df = df.drop('times', axis=1)
df = pd.concat([df, times], axis=1, join_axes=[df.index])
df['streetaddy'] = df.addy.str.split(',').str[0]
df.streetaddy = df.streetaddy.str.replace('Blvd', 'Blvd.')
df.streetaddy = df.streetaddy.str.replace('Blvd..', 'Blvd.')
df.streetaddy = df.streetaddy.str.replace('Avenue', 'Ave')
df.streetaddy = df.streetaddy.str.replace('Ave', 'Ave.')
df.streetaddy = df.streetaddy.str.replace('Ave..', 'Ave.')
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['movieday'] = df['date'].dt.weekday_name
df = df.drop('city', axis=1)
df['cityaddy'] = df.addy.str.split(',').str[1]
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df = df.drop('addy', axis=1)

# Ocala market
oc6_df = df.loc[df['theater'] == 'Ocala Center 6']
ocd_dr = df.loc[df['theater'] == 'Ocala Drive-In']
bv_df = df.loc[df['theater'] == 'Belleview Cinemas']
mt_df = df.loc[df['theater'] == 'Marion Theatre']
omp_df = df.loc[df['theater'] == 'Old Mill Playhouse']
regaloca_df = df.loc[df['theater'] == 'Regal Hollywood Stadium 16 & IMAX - Ocala']
rialto_df = df.loc[df['theater'] == 'Rialto Theatre Spanish Springs Town Square']
amc_df = df.loc[df['theater'] == 'AMC Lake Square 12']
barn_df = df.loc[df['theater'] == 'Barnstorm Theater']

# Gainesville market
celeb_df = df.loc[df['theater'] == 'Celebration Pointe 10']
hipp_df = df.loc[df['theater'] == 'Hippodrome State Theatre']
butler_df = df.loc[df['theater'] == 'Regal Butler Town Center 14']
regalgnv_df = df.loc[df['theater'] == 'Regal Royal Park Stadium 16']
uac_df = df.loc[df['theater'] == 'UA Cinema 90 6']
starke_df = df.loc[df['theater'] == 'Florida Twin']

In [4]:
oc6_df

,date,format,mpaa,stars,synopsis,theater,title,time_1,time_2,time_3,...,time_8,time_9,time_10,time_11,time_12,time_13,time_14,streetaddy,movieday,cityaddy
56,2019-06-30,Standard,R,4.5,John Shaft Jr. may be an FBI cyber security ex...,Ocala Center 6,Shaft (2019),1 p.m.,4 p.m.,7 p.m.,...,None,None,None,None,None,None,None,2021 Silver Springs Blvd.,Sunday,Ocala
91,2019-06-30,Standard,PG,4.5,Aladdin is a lovable street urchin who meets P...,Ocala Center 6,Aladdin (2019),1 p.m.,4 p.m.,7 p.m.,...,None,None,None,None,None,None,None,2021 Silver Springs Blvd.,Sunday,Ocala
152,2019-06-30,Standard,PG,4.5,Max the terrier must cope with some major life...,Ocala Center 6,The Secret Life of Pets 2,1 p.m.,4 p.m.,7 p.m.,...,None,None,None,None,None,None,None,2021 Silver Springs Blvd.,Sunday,Ocala
166,2019-06-29,Standard,PG,4.5,Aladdin is a lovable street urchin who meets P...,Ocala Center 6,Aladdin (2019),1 p.m.,4 p.m.,7 p.m.,...,None,None,None,None,None,None,None,2021 Silver Springs Blvd.,Saturday,Ocala
192,2019-06-29,Standard,PG,4.5,Max the terrier must cope with some major life...,Ocala Center 6,The Secret Life of Pets 2,1 p.m.,4 p.m.,7 p.m.,...,None,None,None,None,None,None,None,2021 Silver Springs Blvd.,Saturday,Ocala
223,2019-06-30,Standard,G,4.5,"Woody, Buzz Lightyear and the rest of the gang...",Ocala Center 6,Toy Story 4,1 p.m.,4 p.m.,7 p.m.,...,None,None,None,None,None,None,None,2021 Silver Springs Blvd.,Sunday,Ocala
225,2019-06-28,Standard,R,4.5,John Shaft Jr. may be an FBI cyber security ex...,Ocala Center 6,Shaft (2019),1 p.m.,4 p.m.,7 p.m.,...,None,None,None,None,None,None,None,2021 Silver Springs Blvd.,Friday,Ocala
240,2019-06-28,Standard,PG,4.5,Aladdin is a lovable street urchin who meets P...,Ocala Center 6,Aladdin (2019),1 p.m.,4 p.m.,7 p.m.,...,None,None,None,None,None,None,None,2021 Silver Springs Blvd.,Friday,Ocala
277,2019-06-28,Standard,R,4,Determined to keep Annabelle from wreaking mor...,Ocala Center 6,Annabelle Comes Home,1 p.m.,4 p.m.,7 p.m.,...,None,None,None,None,None,None,None,2021 Silver Springs Blvd.,Friday,Ocala
293,2019-06-28,Standard,G,4.5,"Woody, Buzz Lightyear and the rest of the gang...",Ocala Center 6,Toy Story 4,1 p.m.,4 p.m.,7 p.m.,...,None,None,None,None,None,None,None,2021 Silver Springs Blvd.,Friday,Ocala


In [7]:
class Ocala6:
    
    """Class for Ocala 6 cinema"""
    
    def __init__(self, oc6_df):
        self.oca6_df = oc6_df
    

In [8]:
a = Ocala6(oc6_df)

In [11]:
print(a)